In [4]:
import pandas as pd

file_path = '../data/Compensation.xlsx'
sheet_names = [str(year) for year in range(2020,2025)]

dfs = [pd.read_excel(file_path, sheet_name = sheet) for sheet in sheet_names]

merged_df = pd.concat(dfs, ignore_index = True)

In [5]:
# 비교할 열 리스트
other_cols = ['장해급여', '간병급여', '유족급여', '장례비', '위로금', '보전비용']

# Case 1: 요양급여 > 0
case1 = (merged_df['요양급여'] > 0).sum()

# Case 2: 요양급여 == 0 & 다른 급여 중 하나라도 > 0
case2 = ((merged_df['요양급여'] == 0) & (merged_df[other_cols].sum(axis=1) > 0)).sum()

# 전체 대비 비율도 확인
total = len(data)

print(f"✅ 요양급여 > 0 : {case1:,}개 ({case1/total*100:.2f}%)")
print(f"✅ 요양급여 = 0 & 다른 급여 중 하나라도 > 0 : {case2:,}개 ({case2/total*100:.2f}%)")


✅ 요양급여 > 0 : 424,557개 (100.09%)
✅ 요양급여 = 0 & 다른 급여 중 하나라도 > 0 : 444개 (0.10%)


In [16]:
filtered_df = merged_df[merged_df[other_cols].sum(axis=1) > 0].copy()
filtered_df

,구분,지역,교육청,학교급,사고자구분,사고자학년,사고자성별,사고시간,사고장소,사고부위,사고형태,사고당시활동,요양급여,장해급여,간병급여,유족급여,장례비,위로금,보전비용
2,F0000003,경남,경상남도교육청,고등학교,일반학생,1학년,남,그 밖의 교육활동 시간,기타 교외,기타,그밖의 손상 사고,기타,0,0,15027050,0,0,0,0
6,F0000007,전북,전라북도교육청,고등학교,일반학생,2학년,남,쉬는시간,운동장,눈,고정된 물체와의 부딪힘,"장난, 놀이",0,320149458,0,0,0,0,0
10,F0000011,충남,아산교육지원청,중학교,체육특기학생,3학년,여,진로활동,강당(체육관),무릎,넘어짐,농구,0,127242595,0,0,0,0,0
17,F0000018,광주,서부교육지원청,초등학교,체육특기학생,5학년,남,진로활동,운동장,아래팔,넘어짐,기타,0,57742520,0,0,0,0,0
34,F0000035,경남,창원교육지원청,중학교,일반학생,1학년,남,식사시간(간식 포함),계단,아래다리(종아리),넘어짐,"장난, 놀이",2881250,37407040,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424789,K0132058,서울,서울특별시교육청,고등학교,일반학생,2학년,남,식사시간(간식 포함),운동장,무릎,고정된 물체와의 부딪힘,축구,266600,44855510,0,0,0,0,0
424821,K0132090,서울,서울특별시교육청,고등학교,체육특기학생,2학년,남,그 밖의 교육활동 시간,운동장,흉부,그밖의 손상 사고,"태권도, 유도, 합기도",0,0,0,515588044,14448100,0,0
424933,K0132202,서울,남부교육지원청,중학교,일반학생,2학년,남,쉬는시간,일반(교과)교실,손가락,넘어짐,기타,102100,125567400,0,0,0,0,0
424985,K0132254,서울,서울특별시교육청,고등학교,일반학생,3학년,남,체육,운동장,무릎,넘어짐,축구,104000,41355800,0,0,0,0,0


In [17]:
filtered_df.isna().sum()

# na 드롭
filtered_df.dropna(inplace=True)


# 필요없는 열 제거
filtered_df.drop(columns=['교육청'], inplace=True)
filtered_df.drop(columns=['구분'], inplace = True)

In [18]:
def age(x):
    a = 0
    if x['학교급'] == '초등학교':
        a = 7
    elif x['학교급'] == '중학교':
        a = 13
    else:
        a = 16
    return a

def age2(y):
    b = 0
    if y['사고자학년'] == '1학년':
        b = y['추정나이'] + 0
    elif y['사고자학년'] == '2학년':
        b = y['추정나이'] + 1
    elif y['사고자학년'] == '3학년':
        b = y['추정나이'] + 2
    elif y['사고자학년'] == '4학년':
        b = y['추정나이'] + 3
    elif y['사고자학년'] == '5학년':
        b = y['추정나이'] + 4
    elif y['사고자학년'] == '6학년':
        b = y['추정나이'] + 5
    return b

filtered_df['추정나이'] = filtered_df.apply(age, axis=1)
filtered_df['추정나이'] = filtered_df.apply(age2, axis=1)

In [19]:
col_to_sum = ['요양급여', '장해급여', '간병급여', '유족급여', '장례비', '위로금', '보전비용']

filtered_df['보상급여'] = filtered_df[col_to_sum].sum(axis=1)
filtered_df.drop(columns=col_to_sum, inplace=True)

filtered_df

,지역,학교급,사고자구분,사고자학년,사고자성별,사고시간,사고장소,사고부위,사고형태,사고당시활동,추정나이,보상급여
2,경남,고등학교,일반학생,1학년,남,그 밖의 교육활동 시간,기타 교외,기타,그밖의 손상 사고,기타,16,15027050
6,전북,고등학교,일반학생,2학년,남,쉬는시간,운동장,눈,고정된 물체와의 부딪힘,"장난, 놀이",17,320149458
10,충남,중학교,체육특기학생,3학년,여,진로활동,강당(체육관),무릎,넘어짐,농구,15,127242595
17,광주,초등학교,체육특기학생,5학년,남,진로활동,운동장,아래팔,넘어짐,기타,11,57742520
34,경남,중학교,일반학생,1학년,남,식사시간(간식 포함),계단,아래다리(종아리),넘어짐,"장난, 놀이",13,40288290
...,...,...,...,...,...,...,...,...,...,...,...,...
424789,서울,고등학교,일반학생,2학년,남,식사시간(간식 포함),운동장,무릎,고정된 물체와의 부딪힘,축구,17,45122110
424821,서울,고등학교,체육특기학생,2학년,남,그 밖의 교육활동 시간,운동장,흉부,그밖의 손상 사고,"태권도, 유도, 합기도",17,530036144
424933,서울,중학교,일반학생,2학년,남,쉬는시간,일반(교과)교실,손가락,넘어짐,기타,14,125669500
424985,서울,고등학교,일반학생,3학년,남,체육,운동장,무릎,넘어짐,축구,18,41459800
